Make sure we have a GPU :

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

# Fetch dataset

In [ ]:
!git clone https://github.com/pekatour/Charlie-DB.git
!mkdir datasets
!cp -r ./Charlie-DB/dataset datasets/dataset
!cp ./Charlie-DB/data.yaml data.yaml
!pip install ultralytics

In [ ]:
!ls datasets/dataset

Run this only if you want to reimport the dataset

In [ ]:
!rm -rf datasets/
!mkdir datasets

# Train

In [ ]:
# Import necessary libraries
from ultralytics import YOLO

# Load a pretrained YOLO model
model = YOLO('yolo11n.pt') # Or any other suitable model

# Train the model
results = model.train(data='./data.yaml', epochs=50, imgsz=300) # Adjust parameters as needed

In [ ]:
# Résultats
metrics = model.val()
print(metrics.box.map)

# Check

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")

# Run inference on test images and save results
results = model.predict(source="./datasets/dataset/test/images", save=True)

In [ ]:
import glob
import random
import os
from IPython.display import Image, display

directory = './runs/detect/predict'
# List all files in the directory and filter for .jpg files
image_files = [f for f in os.listdir(directory) if f.endswith('.jpg')]

# Get full paths
image_paths = [os.path.join(directory, f) for f in image_files]

sample_size = min(10, len(image_paths))
for image_path in random.sample(image_paths, sample_size):
  display(Image(filename=image_path, height=400))
  print('\n')

In [ ]:
%load_ext autoreload
%autoreload 2

## Predict on a full image

In [ ]:
from full_image_predict import full_image_predict

full_image_predict(model, "./original-images/5-9-0.jpg", 300)

# Export

In [ ]:
from google.colab import files
!zip -r train.zip runs/detect/train/
files.download('train.zip')

In [ ]:
# Optionnel
model.export(format="onnx")